In [ ]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())  # 检查是否支持 Apple MPS 加速


# 在 Notebook 中运行 BLIP-2 + MedLLaMA 2

## 1. 加载 BLIP-2

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image

device = "mps" if torch.backends.mps.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float32)

model.to(device)